In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pytickersymbols import PyTickerSymbols

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
# Download stock data using yfinance
def get_stock_data(ticker, start, end):
    data = yf.download(ticker, start=start, end=end)
    return data

In [ ]:
# Calculate RSI manually using pandas
def calculate_rsi(data, period=14):
    delta = data['Adj Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))
    return data

In [ ]:
# Implement the trading strategy with stop-sell condition
def trading_strategy(data):
    position = 0  # 1 means we own the stock, 0 means we don't
    holding_period = 0  # Track how long we’ve held the position
    max_holding_days = 5  # Stop-sell after 5 days
    buy_price = 0
    data['Signal'] = 0

    for i in range(1, len(data)):
        if data['RSI'][i-1] < 51 and data['RSI'][i] >= 51 and position == 0:
            # Buy signal: RSI crosses from below 51 to above 51
            position = 1
            holding_period = 1  # Reset holding period counter on buy
            buy_price = data['Adj Close'][i]
            data['Signal'][i] = 1  # 1 for buy
            print(f"Bought at {buy_price} on {data.index[i]}")
        
        elif position == 1:
            holding_period += 1
            
            # Sell signal: RSI crosses from below 71 to above 71 or stop-sell after 5 days
            if (data['RSI'][i-1] < 71 and data['RSI'][i] >= 71) or holding_period > max_holding_days:
                position = 0
                sell_price = data['Adj Close'][i]
                data['Signal'][i] = -1  # -1 for sell
                print(f"Sold at {sell_price} on {data.index[i]} (held for {holding_period - 1} days)")
                holding_period = 0  # Reset the holding period after selling
    
    return data

In [ ]:
# Backtest the strategy
def backtest(data):
    # Create Position column where we hold the stock only between buy and sell signals
    data['Position'] = 0  # Initialize the position as no stock held

    for i in range(1, len(data)):
        if data['Signal'][i] == 1:  # Buy signal
            data['Position'][i] = 1
        elif data['Signal'][i] == -1:  # Sell signal
            data['Position'][i] = 0
        else:
            data['Position'][i] = data['Position'][i-1]  # Continue holding or not

    # Calculate daily returns
    data['Daily Return'] = data['Adj Close'].pct_change()
    
    # Strategy returns only when we're in position
    data['Strategy Return'] = data['Daily Return'] * data['Position'].shift(1)
    
    # Set returns to 0 for days when not holding a position
    data['Strategy Return'].fillna(0, inplace=True)

    # Calculate cumulative returns
    data['Cumulative Market Return'] = (1 + data['Daily Return']).cumprod()
    data['Cumulative Strategy Return'] = (1 + data['Strategy Return']).cumprod()

    total_strategy_return = (data['Cumulative Strategy Return'].iloc[-1] * 100) - 1
    total_strategy_return = "{:.2f}%".format(total_strategy_return)

    return total_strategy_return

In [ ]:
# Plot the results
def plot_results(data):
    plt.figure(figsize=(14, 7))
    plt.plot(data['Cumulative Market Return'], label='Market Return')
    plt.plot(data['Cumulative Strategy Return'], label='Strategy Return', linestyle='--')
    plt.title('Market vs Strategy Performance')
    plt.legend()
    plt.show()

In [ ]:
#  Single Main function
def main_single():
    stock_data = PyTickerSymbols()
    sp500_symbols = stock_data.get_stocks_by_index('S&P 500')
    #symbols = [stock['symbol'] for stock in sp500_symbols]  # S&P 500 stock symbols
    #symbols = ['AAPL', 'NVDA']
    symbol = 'AAPL'

    #returns= pd.DataFrame()

    start = '2023-10-17'
    end = '2024-10-17'
    
    data = get_stock_data(symbol, start, end)
    data = get_stock_data(symbol, start, end)
    data = calculate_rsi(data)
    data = trading_strategy(data)
    data = backtest(data)
    #print(data)

    #returns[symbol] = data

    return data
main_single()

In [ ]:
#df.to_csv('temp test.csv', index=True)